Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import time
import random as rd
import os
import math

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset'][:200000]
  train_labels = save['train_labels'][:200000]
  valid_dataset = save['valid_dataset'][:5000]
  valid_labels = save['valid_labels'][:5000]
  test_dataset = save['test_dataset'][:5000]
  test_labels = save['test_labels'][:5000]
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (5000, 28, 28) (5000,)
Test set (5000, 28, 28) (5000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (5000, 28, 28, 1) (5000, 10)
Test set (5000, 28, 28, 1) (5000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
def run(graph, num_steps = 1001):
    with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print('Initialized')
      for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 200 == 0):
          print('Minibatch loss at step %d: %f' % (step, l))
          print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
          print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

In [6]:
batch_size = 16
patch_size = 5
depth = 4
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
  run(graph)

Initialized
Minibatch loss at step 0: 2.420867
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 200: 0.709268
Minibatch accuracy: 81.2%
Validation accuracy: 76.8%
Minibatch loss at step 400: 0.652535
Minibatch accuracy: 75.0%
Validation accuracy: 78.9%
Minibatch loss at step 600: 0.593438
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 800: 0.548581
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 0.236765
Minibatch accuracy: 93.8%
Validation accuracy: 82.6%
Test accuracy: 89.1%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [8]:
batch_size = 16
patch_size = 5
depth = 4
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
  run(graph)

Initialized
Minibatch loss at step 0: 3.349397
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 200: 0.786014
Minibatch accuracy: 81.2%
Validation accuracy: 71.3%
Minibatch loss at step 400: 0.751492
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
Minibatch loss at step 600: 0.578230
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 800: 0.811632
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 1000: 0.157112
Minibatch accuracy: 93.8%
Validation accuracy: 84.0%
Test accuracy: 89.8%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [5]:
 def file_name(epoch_nb,test_acc_log, num_epochs, batch_size,learning_rate, decay_rate, \
              dropout_keep_prob, l2_penalty, train_dataset,train_labels, path_to_save):
    file_name = str(epoch_nb)+'_'+str(test_acc_log) + '_ne.'+str(num_epochs)+'_bs.' + str(batch_size)+ \
    '_lr.'+str(learning_rate) + '_dr'+str(decay_rate) + '_d.' + str(dropout_keep_prob) + \
    '_l2.'+ str(l2_penalty) + '_t'+'.csv'
    file_name = os.path.join(path_to_save, file_name)
    return file_name

In [6]:
def run_model(num_epochs, batch_size, learning_rate, decay_rate, dropout_keep_prob, l2_penalty, \
                train_dataset,train_labels, path_to_save, quiet = True, epoch_calc = False, beta1=0.9, beta2=0.999):
    num_steps = (train_labels.shape[0] * num_epochs) / batch_size
    epoch_log = np.full(num_steps, np.nan)
    loss_log = np.full(num_steps, np.nan)
    acc_log = np.full(num_steps, np.nan)
    val_acc_log = np.full(num_steps, np.nan)
    test_acc_log = np.full(num_steps, np.nan)
    print(num_steps)
    if not os.path.exists(path_to_save):
        os.makedirs(path_to_save)
        
    epoch_nb = -1
    graph = tf.Graph()
    # GRAPH
    with graph.as_default():
      # Input parameters:
      tf_learning_rate_0 = tf.placeholder(tf.float32)
      tf_dropout_keep_prob =  tf.placeholder(tf.float32)
      tf_l2_penalty =  tf.placeholder(tf.float32)
      tf_decay_rate = tf.placeholder(tf.float32)
      tf_step_nb = tf.placeholder(tf.float32)  
      # Input data.
      tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      #C1
      # 5x5 filters with depth 6
      layer1_weights = tf.Variable(tf.truncated_normal(
          [5, 5, num_channels, 6], stddev=0.1))
      layer1_biases = tf.Variable(tf.zeros([6]))
      #C3
      # 5x5 filters with depth 6
      layer3_weights = tf.Variable(tf.truncated_normal(
          [5, 5, 6, 16], stddev=0.1))
      layer3_biases = tf.Variable(tf.constant(1.0, shape=[16]))
      #C5
      # 5x5 filters with depth 120
      layer5_weights = tf.Variable(tf.truncated_normal(
          [5, 5, 16, 120], stddev= 0.1))
      layer5_biases = tf.Variable(tf.constant(1.0, shape=[120]))
      #F6 
      #84 fully connected
      layer6_weights = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * 120, 84], stddev= 0.1))
      layer6_biases = tf.Variable(tf.constant(1.0, shape=[84]))
      # final 
      layer7_weights = tf.Variable(tf.truncated_normal(
          [84, num_labels],  stddev= 0.1))
      layer7_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

      # Model.
      def model(data):
        #C1
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        #S2
        pool = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
        #C3
        conv = tf.nn.conv2d(pool, layer3_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer3_biases)
        #S4
        pool = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
        #C5
        conv = tf.nn.conv2d(pool, layer5_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer5_biases)
        #F6
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer6_weights) + layer6_biases)
        return tf.matmul(hidden, layer7_weights) + layer7_biases

      # Training computation.
      logits = model(tf_train_dataset)
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
      #reg = tf_l2_penalty*(tf.nn.l2_loss(layer1_weights)+tf.nn.l2_loss(layer3_weights)+
      #                     tf.nn.l2_loss(layer5_weights)+tf.nn.l2_loss(layer6_weights)+
      #                     tf.nn.l2_loss(layer7_weights))
      #loss+=reg
        
      # Optimizer.
      tf_learning_rate = tf.train.exponential_decay(tf_learning_rate_0, tf_step_nb, 3125, tf_decay_rate, staircase = True)
      optimizer = tf.train.AdamOptimizer(tf_learning_rate, beta1,beta2).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
      test_prediction = tf.nn.softmax(model(tf_test_dataset))
 #starting new session 
    epoch_nb = -1
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, 
                        tf_learning_rate_0 : learning_rate, tf_dropout_keep_prob : dropout_keep_prob, 
                         tf_l2_penalty : l2_penalty, tf_decay_rate : decay_rate, tf_step_nb : step}
            _, l, predictions = session.run(
                    [optimizer, loss, train_prediction], feed_dict=feed_dict)
            loss_log[step] = l
            epoch_log[step] = step * batch_size / float(train_labels.shape[0])
            if (step % 100 == 0):
                val_acc_log[step] = accuracy(valid_prediction.eval(), valid_labels)
                acc_log[step] = accuracy(predictions, batch_labels)
                if not quiet: 
                    print("Minibatch loss at step %d: %f" % (step, l))
                    print("Minibatch accuracy: %.1f%%" % acc_log[step])
                    print("Validation accuracy: %.1f%%" % val_acc_log[step])
            if epoch_calc and epoch_log[step] > epoch_nb:
                epoch_nb+=1
                test_acc_log[step] = accuracy(test_prediction.eval(), test_labels)
                val_acc_log[step] = accuracy(valid_prediction.eval(), valid_labels)
                acc_log[step] = accuracy(predictions, batch_labels)
                print("Test accuracy: %.1f%%" %  test_acc_log[step])
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % acc_log[step])
                print("Validation accuracy: %.1f%%" % val_acc_log[step])
                fn = file_name(epoch_nb,test_acc_log[step], num_epochs, batch_size, \
                               learning_rate, decay_rate, dropout_keep_prob, l2_penalty, \
                               train_dataset,train_labels, path_to_save)
                np.savetxt(fn, np.vstack((epoch_log, loss_log, acc_log, val_acc_log, test_acc_log)).T, 
                   delimiter = ';', fmt='%.10f')
        
        test_acc_log[step] = accuracy(test_prediction.eval(), test_labels)
        fn = file_name(epoch_nb,test_acc_log[step], num_epochs, batch_size, \
                               learning_rate, decay_rate, dropout_keep_prob, l2_penalty, \
                               train_dataset,train_labels, path_to_save)
        np.savetxt(fn, np.vstack((epoch_log, loss_log, acc_log, val_acc_log, test_acc_log)).T, 
                   delimiter = ';', fmt='%.10f')
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [ ]:
num_epochs = 1
for i in range(5):
    t0 = time.time()
    batch_size = 16
    learning_rate = 10 ** rd.uniform(-6., 0)
    decay_rate = rd.uniform(0.9,1)
    dropout_keep_prob = rd.uniform(0.2,1)
    l2_penalty = 10 ** rd.uniform(-6., 1.)
    print('Model',i,'batch_size:',batch_size, 'learning_rate:', learning_rate,\
          'decay_rate:', decay_rate, 'dropout_keep_prob:', dropout_keep_prob,
          'l2_penalty:', l2_penalty)
    run_model(num_epochs, batch_size, learning_rate, decay_rate, dropout_keep_prob, l2_penalty, \
                train_dataset,train_labels, '/notebooks/run10', quiet = False, epoch_calc = True)   
    t1 = time.time()
    print('Model',i,'- time:',(t1-t0)/60, 'min.')

In [7]:
##### t0 = time.time()
num_epochs = 7
batch_size = 64
learning_rate = 0.001
decay_rate = 0.5
dropout_keep_prob = 0.85
l2_penalty = 0

run_model(num_epochs, batch_size, learning_rate, decay_rate, dropout_keep_prob, l2_penalty, \
            train_dataset,train_labels, '/notebooks/run5', quiet = False, epoch_calc = True)   
#t1 = time.time()
#print('Model',i,'- time:',(t1-t0)/60, 'min.')

21875
Minibatch loss at step 0: 10.452844
Minibatch accuracy: 12.5%
Validation accuracy: 9.7%
Test accuracy: 10.2%
Minibatch loss at step 0: 10.452844
Minibatch accuracy: 12.5%
Validation accuracy: 9.7%
Test accuracy: 7.8%
Minibatch loss at step 1: 17.926678
Minibatch accuracy: 10.9%
Validation accuracy: 7.7%
Minibatch loss at step 100: 1.203467
Minibatch accuracy: 64.1%
Validation accuracy: 69.2%
Minibatch loss at step 200: 0.718723
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 300: 0.857965
Minibatch accuracy: 78.1%
Validation accuracy: 80.1%
Minibatch loss at step 400: 0.746172
Minibatch accuracy: 78.1%
Validation accuracy: 82.1%
Minibatch loss at step 500: 0.678193
Minibatch accuracy: 81.2%
Validation accuracy: 83.2%
Minibatch loss at step 600: 0.338664
Minibatch accuracy: 89.1%
Validation accuracy: 83.6%
Minibatch loss at step 700: 0.450690
Minibatch accuracy: 84.4%
Validation accuracy: 85.1%
Minibatch loss at step 800: 0.289012
Minibatch accuracy: 92